## Data aggregation

In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import arrow
from collections import Counter, OrderedDict
from datetime import datetime
from bokeh.plotting import figure, output_notebook, show
from bokeh.charts import Area, vplot, defaults
from bokeh.models import Range1d
from bokeh.palettes import brewer, Spectral11
from bokeh.models.mappers import LinearColorMapper
import numpy as np

First we grab the account activation and deactivation data off of the official WMF wiki and crunch that.

In [2]:
deactivation_soup = BeautifulSoup(requests.get('https://wikimediafoundation.org/w/index.php?title=Special:Log&offset=&limit=500&type=block&user=&page=&tagfilter=&hide_thanks_log=1&hide_tag_log=1').text, 'html.parser')

In [3]:
# for logline in [li.text for li in deactivation_soup.find_all('li') if 'class' in li.attrs  and li.get('class')[0] == 'mw-logline-block']:
#     print(logline)

In [4]:
activation_soup = BeautifulSoup(requests.get('https://wikimediafoundation.org/w/index.php?title=Special:Log&offset=&limit=500&type=newusers&user=&page=&tagfilter=').text, 'html.parser')

In [5]:
# for logline in [li.text for li in activation_soup.find_all('li') if 'class' in li.attrs  and li.get('class')[0] == 'mw-logline-newusers']:
#     print(logline)

In [6]:
# Create activation staff DataFrame from activation log.

dates = []
names = []
for logline in [li.text for li in activation_soup.find_all('li') if 'class' in li.attrs  and li.get('class')[0] == 'mw-logline-newusers' and 'was created' in li.text]:
    date = ' '.join(logline.split(' ')[2:5])
#     print(date)
    date = arrow.get(date, 'D MMMM YYYY')
    dates.append(date)
    name = logline[logline.find("User account") + 13:logline.find("(talk") - 1]
#     print(name)
    names.append(name)

In [7]:
staff_activations = DataFrame(data={'Joined': dates}, index=names)
# staff_activations = DataFrame(data={'Name': names, 'Joined': dates})
# staff_activations

In [9]:
# Create deactivation staff DataFrame from activation log.

dates = []
names = []

for logline in [li.text for li in deactivation_soup.find_all('li') if 'class' in li.attrs  and li.get('class')[0] == 'mw-logline-block' and 'deactivated' in li.text]:
#     print(logline[logline.find(",") + 2:], "SPACE", logline[logline.find(",") + 2:logline.rfind("(talk")])
    date = ' '.join(logline.split(' ')[2:5])
#     print(date)
    date = arrow.get(date, 'D MMMM YYYY')
    dates.append(date)
    name = logline[logline.find('deactivated') + 12:]
    name = name[:name.find("(talk") - 1]
#     print(name)
    names.append(name)

In [10]:
staff_deactivations = DataFrame(index=names, data={'Left': dates})
# staff_deactivations = DataFrame(data={'Name': names, 'Left': dates})
# staff_deactivations

In [12]:
# staff = pd.concat([staff_activations, staff_deactivations])
# staff = staff_activations.append(staff_deactivations)
# staff = pd.merge(staff_activations, staff_deactivations, how='inner')
staff = staff_activations.join(staff_deactivations)

In [13]:
staff.sort_values(by = "Left", ascending=False)

,Joined,Left
AKoval (WMF),2013-07-17T00:00:00+00:00,2016-02-16T00:00:00+00:00
ACella (WMF),2015-07-06T00:00:00+00:00,2016-02-10T00:00:00+00:00
HZhang (WMF),2015-06-08T00:00:00+00:00,2016-02-10T00:00:00+00:00
AKoval (WMF),2013-07-17T00:00:00+00:00,2016-02-10T00:00:00+00:00
JChojnacki (WMF),2015-12-02T00:00:00+00:00,2016-02-10T00:00:00+00:00
JLiong (WMF),2015-05-27T00:00:00+00:00,2016-02-10T00:00:00+00:00
DHamilton (WMF),2015-06-10T00:00:00+00:00,2016-02-10T00:00:00+00:00
JUnikowski (WMF),2015-08-24T00:00:00+00:00,2016-02-10T00:00:00+00:00
LMartinez (WMF),2013-07-10T00:00:00+00:00,2016-02-10T00:00:00+00:00
BSher (WMF),2015-05-15T00:00:00+00:00,2016-02-10T00:00:00+00:00


Data is still dirtied by the presence of users who are not WMF staffers in the staff creation listings. There is no easy way to automatically check whether or not an account without a (WMF) at the end is a staff account or not (which is why that policy was eventually instituted in the first place), so we will simply drop the names that we're not sure of from the table.

To distinguish this from our complete table we'll call and work with `filtered_data` going forward.

Obviously accounts that later on got closed are those of WMF staffers (volunteer accounts are never closed). The remaining trouble spots still contain a number of extremely senior WMF staffers, but unfortunately they cannot really be accounted for.

Given the way that the data is presented---we hand-roll the data for the two executives too senior to be included in this data---this does not cause us any issues in this analysis.

Note also that there is at least one case, that of Anna Koval, in which the account was deactivated (and, for the matter, activated) not once but *twice*. This results in duplicate entries.

In [15]:
bad_names = [name for name in staff.index if 'WMF' not in name and str(staff.ix[name, 'Left']) == 'nan']
filtered_staff = staff.drop(bad_names)

## Data munging

With the data in place let's start processing it into something informative. First let's get turnover data.

### Absolute turnover

In [16]:
joined_count_by_month = Counter()
for date in filtered_staff['Joined']:
    joined_count_by_month.update([date.floor('month')])

In [17]:
# joined_count_by_month.most_common()

In [18]:
left_count_by_month = Counter()
for date in filtered_staff['Left']:
    if str(date) != 'nan':
        left_count_by_month.update([date.floor('month')])

In [19]:
# left_count_by_month.most_common()

In [20]:
time_index = [date.datetime for date in arrow.Arrow.range('month', arrow.get(2011, 8, 1), arrow.Arrow(2016, 2, 1).floor('month'))]
# time_index

In [21]:
join_df = DataFrame(index=time_index, data={'Joined': joined_count_by_month})
left_df = DataFrame(index=time_index, data={'Left': left_count_by_month})
left_df = left_df.fillna(0)

In [22]:
output_notebook(hide_banner=True)

p = figure(plot_width=800,
           plot_height=400,
           y_range=[0, 37],
           title='Turnover at the Wikimedia Foundation',
           title_text_font_size="18px",
           x_axis_type="datetime"
)

p.line(
    join_df.index,
    join_df['Joined'],
    line_width=2,
    line_color='#339966'
)

# p.patch(
#     x=join_df.index,
#     y=join_df['Joined'],
#     fill_color='#339966',
#     line_color='#339966'
# )

p.line(
    left_df.index,
    left_df['Left'],
    line_width=2,
    line_color='#990000'
)

p.line(
    [arrow.Arrow(2014,6,1).datetime, arrow.Arrow(2014,6,1).datetime],
    [0, 37],
    line_width=1,
    line_color='black'
)

show(p)

The large spike the WMF HR people catching up with a number of accounts which were not closed properly at the time of their leaving the Foundation. These all occured during Tretikov's time, however. To percieve the before and after better we can average the before and after, as this graph is rather uninformative taken on its own.

In [23]:
sue_out, tretikov_out = left_df[left_df.index <= arrow.Arrow(2014,6,1).datetime], left_df[left_df.index > arrow.Arrow(2014,6,1).datetime]

Calculate average staff goings per month. This is an artificial statistic: remember that the WMF wasn't always the size it is today.

In [24]:
sue_out['Left'].mean(), tretikov_out['Left'].mean()

(2.657142857142857, 5.0)

As of writing the WMF has 293 employees. Calculate yearly attrition rate.

In [25]:
(12*tretikov_out['Left'].mean())/293

0.20477815699658702

How about the hire rate?

In [26]:
sue_in, tretikov_in = join_df[join_df.index <= arrow.Arrow(2014,6,1).datetime], join_df[join_df.index > arrow.Arrow(2014,6,1).datetime]

sue_in['Joined'].mean(), tretikov_in['Joined'].mean()

(6.428571428571429, 8.65)

In [27]:
(12*tretikov_in['Joined'].mean())/293

0.3542662116040956

In other words, under Tretikov's leadership the WMF has experienced a **20%** attrition rate, losing **60** people a year, and a **35%** hire rate, gaining **100** people per year. The result is **40 more employees per year**, an average of **+15% year-on-year growth**.

For reference, estimated turnover rates for technology companies are estimated to range from **20% to 30%**.

### Still here

Another way to visualize this is to examine who's still here.

In [28]:
still_in = Counter()
for month in time_index:
    still_in.update({month: 0})
for row in filtered_staff.iterrows():
    if str(row[1]['Left']) == 'nan':
        still_in.update([row[1]['Joined'].floor('month').datetime])

In [59]:
# still_in

In [30]:
still_in_cumulative = np.cumsum(list(still_in.values()))
still_in_cumulative = still_in_cumulative + 293 - 205

In [31]:
still_in_cumulative_percentage = still_in_cumulative / 293

In [32]:
output_notebook(hide_banner=True)

p = figure(plot_width=800,
           plot_height=400,
           y_range = [0, 1],
           title='Cumulative employment by date at the Wikimedia Foundation',
           title_text_font_size="18px",
           x_axis_type="datetime"
)

p.line(
    time_index,
    still_in_cumulative_percentage,
    line_width=2,
    line_color='#006699'
)

p.line(
    [arrow.Arrow(2014,6,1).datetime, arrow.Arrow(2014,6,1).datetime],
    [0, 1],
    line_width=1,
    line_color='black'
)

show(p)

### Executive leadership

To measure the relative experience level at the top of the Wikimedia Foundation, we plot the experiential levels of current C-suite executives over time.

The list of directors used here was hand-generated, as some go so far back that the scrape did not find anything for him.

In [33]:
names = ['GByrd (WMF)', 'Gyoung', 'Lgruwell', 'ASengupta (WMF)', 'LilaTretikov (WMF)', 'Damon Sicore (WMF)',
             'Katherine (WMF)', 'LuisV (WMF)', 'TGilbey (WMF)', 'KKarimkhany (WMF)', 'WMoran (WMF)', ]
joined_dates = []
left_dates = []

for name in names:
    _exec = staff[staff.index == name]
    joined_dates.append(_exec['Joined'][0].datetime)
    if str(_exec['Left'][0]) != 'nan':
        left_dates.append(_exec['Left'][0].datetime)
    else:
        left_dates.append(np.nan)
# Gruwell became CRO in September 2013.
# Moran was briefly not C-level.
# Villa spent a while as deputy legal head.

# Hand-spin some of this.
names += ['Erik Moeller (WMF)', 'Geoff Brigham (WMF)', 'Jaime Villagomez (WMF)', 'Boryana Dineva (WMF)']
joined_dates += [arrow.Arrow(2007, 12, 1).datetime, arrow.Arrow(2011, 3, 1).datetime, arrow.Arrow(2016, 1, 1).datetime, arrow.Arrow(2015, 10, 1).datetime]
left_dates += [arrow.Arrow(2015, 4, 1).datetime, np.nan, np.nan, np.nan]
# Fix Luis leaving not being in the data yet.
left_dates[7] = arrow.Arrow(2016, 2, 1).datetime
# I am not counting Maggie Davis's position as interim officer because of the interimicty.

In [34]:
execs = DataFrame(index=names, data={'Joined': joined_dates, 'Left': left_dates})

In [35]:
execs

,Joined,Left
GByrd (WMF),2011-08-02 00:00:00+00:00,2015-10-05 00:00:00+00:00
Gyoung,2011-12-30 00:00:00+00:00,2015-08-31 00:00:00+00:00
Lgruwell,2011-09-07 00:00:00+00:00,NaT
ASengupta (WMF),2012-07-02 00:00:00+00:00,2015-08-31 00:00:00+00:00
LilaTretikov (WMF),2014-05-01 00:00:00+00:00,NaT
Damon Sicore (WMF),2014-09-29 00:00:00+00:00,2015-06-16 00:00:00+00:00
Katherine (WMF),2014-04-14 00:00:00+00:00,NaT
LuisV (WMF),2013-02-13 00:00:00+00:00,2016-02-01 00:00:00+00:00
TGilbey (WMF),2015-01-27 00:00:00+00:00,2015-12-14 00:00:00+00:00
KKarimkhany (WMF),2015-03-27 00:00:00+00:00,2015-08-31 00:00:00+00:00


In [36]:
tretikov_time_index = [date.datetime for date in arrow.Arrow.range('month', arrow.get(2014, 7, 1), arrow.Arrow(2016, 3, 1).floor('month'))]

In [37]:
execs_dat = OrderedDict()
for i in range(len(names)):
    dat = []
    for time in tretikov_time_index:
        if joined_dates[i] > time or (str(left_dates[i]) != 'nan' and left_dates[i] < time):
            dat.append(0)
        else:
            dat.append((time - joined_dates[i]).days)
    execs_dat.update({names[i]: dat})

The first version of the graphic that I created used data on the y-axis in days, however on reflection it would be clearer to use data in years. To get this, we use the heuristic value `days / 365`.

In [76]:
for i in range(len(names)):
    dat = []
    for time in tretikov_time_index:
        if joined_dates[i] > time or (str(left_dates[i]) != 'nan' and left_dates[i] < time):
            dat.append(0)
        else:
#             dat.append(round((time - joined_dates[i]).days / 30)) # months
#             dat.append((time - joined_dates[i]).days) # days
            dat.append((time - joined_dates[i]).days / 365) # years
    execs_dat.update({names[i]: dat})

In [77]:
execs_dat_sorted = OrderedDict(sorted(execs_dat.items(), key=lambda item: item[1], reverse=True))

In [78]:
execs_dat_sorted

OrderedDict([('Erik Moeller (WMF)',
              [6.586301369863014,
               6.671232876712328,
               6.756164383561644,
               6.838356164383562,
               6.923287671232877,
               7.005479452054795,
               7.090410958904109,
               7.175342465753425,
               7.252054794520548,
               7.336986301369863,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0]),
             ('Geoff Brigham (WMF)',
              [3.336986301369863,
               3.421917808219178,
               3.506849315068493,
               3.589041095890411,
               3.673972602739726,
               3.756164383561644,
               3.841095890410959,
               3.926027397260274,
               4.002739726027397,
               4.087671232876712,
               4.16986301369863,
              

In [83]:
output_notebook(hide_banner=True)

# data = dict(
#     python=[2, 3, 7, 5, 26, 221, 44, 233, 254, 265, 266, 267, 120, 111],
#     pypy=[12, 33, 47, 15, 126, 121, 144, 233, 254, 225, 226, 267, 110, 130],
#     jython=[22, 43, 10, 25, 26, 101, 114, 203, 194, 215, 201, 227, 139, 160],
# )

data = execs_dat_sorted

# colormap = LinearColorMapper(low=0, high=2500, palette=brewer['Blues'][9])

area = Area(data,
            title="Cumulative executive experience at the Wikimedia Foundation",
            stack=True,
            xlabel='',
            ylabel='',
            plot_width=800,
            plot_height=400,
#             palette=['black', 'white']
            palette = brewer["Reds"][8]
           )

area.x_range = Range1d(start=0, end=20)
area.y_range = Range1d(start=0, end=30)
# area.fill_color = colormap
# area.palette = brewer["Reds"][8]

show(area)

# above, background_fill_alpha, background_fill_color, below, border_fill_alpha, border_fill_color, disabled,
# extra_x_ranges, extra_y_ranges, h_symmetry, hidpi, left, legend, lod_factor, lod_interval, lod_threshold,
# lod_timeout, logo, min_border, min_border_bottom, min_border_left, min_border_right, min_border_top, name,
# outline_line_alpha, outline_line_cap, outline_line_color, outline_line_dash, outline_line_dash_offset,
# outline_line_join, outline_line_width, plot_height, plot_width, renderers, responsive, right, tags, title,
# title_standoff, title_text_align, title_text_alpha, title_text_baseline, title_text_color, title_text_font,
# title_text_font_size, title_text_font_style, tool_events, toolbar_location, tools, v_symmetry, webgl,
# x_mapper_type, x_range, xgrid, xlabel, xscale, y_mapper_type, y_range, ygrid, ylabel or yscale

In [42]:
# # Organized name, title, join date, exec date, leave date.
# [['Erik Moeller', 'Deputy Director', arrow.Arrow(2007, 12, 1).datetime, arrow.Arrow(2007, 12, 1).datetime, arrow.Arrow(2015, 4, 1).datetime],
#  ['Anasuya Sengupta', 'Senior Director of Grantmaking', arrow.Arrow(2012, 7, 1).datetime, arrow.Arrow(2012, 7, 1).datetime, arrow.Arrow(2015, 8, 1).datetime],
#  ['Lisa Gruwell', 'Chief Revenue Officer', arrow.Arrow(2011, 9, 1).datetime, np.nan]
#  ]